In [32]:
import requests
from bs4 import BeautifulSoup
import time
import re
from tqdm import tqdm

# 페이지 번호 생성하는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)

# 검색 URL 생성하는 함수
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성된 URL: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성된 URLs: ", urls)
        return urls    
    
## 텍스트 전처리 사용자 정의함수(UDF of text pre-processing)
def text_preprocessor(s):
    import re
    
    ## (1) [], (), {}, <> 괄호와 괄호 안 문자 제거하기
    pattern = r'\([^)]*\)'  # ()
    s = re.sub(pattern=pattern, repl='', string=s)
    
    pattern = r'\[[^)]*\]'  # []
    s = re.sub(pattern=pattern, repl='', string=s)
    
    pattern = r'\<[^)]*\>'  # <>
    s = re.sub(pattern=pattern, repl='', string=s)
    
    pattern = r'\{[^)]*\}'  # {}
    s = re.sub(pattern=pattern, repl='', string=s)
    
    ## (2) '...외', '...총' 제거하기
    s = s.replace('...외', ' ')
    s = s.replace('...총', ' ')
    
    ## (3) 특수문자 제거
    pattern = r'[^a-zA-Z가-힣]'
    s = re.sub(pattern=pattern, repl=' ', string=s)
    
    ## (4) 단위 제거: cm, km, etc.
    units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
    for unit in units:
        s = s.lower() # 대문자를 소문자로 변환
        s = s.replace(unit, '')
        
    # (5) 공백 기준으로 분할하기
    s_split = s.split()
    
    # (6) 글자 1개만 있으면 제외하기
    s_list = []
    for word in s_split:
        if len(word) !=1:
            s_list.append(word)
            
    return s_list

# 검색어와 페이지 입력 받기
search = input("검색어를 입력하세요:")
start_page = int(input("\n크롤링을 시작할 페이지를 :"))
end_page = int(input("\n크롤링을 종료할 페이지를 :"))

# Naver 검색 URL 생성
urls = makeUrl(search, start_page, end_page)

# 기사 URL 크롤링
article_urls = []
for url in urls:
    article_urls.extend(articles_crawler(url))

# Naver 뉴스 URL 필터링
final_urls = [url for url in article_urls if "news.naver.com" in url]

# 데이터 저장을 위한 리스트 초기화
news_titles = []
news_contents = []

# 뉴스 내용 추출하는 함수
def extract_news_content(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    news = requests.get(url, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title is None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    content = news_html.select("article#dic_area")
    if not content:
        content = news_html.select("#articleBody")
    title = re.sub(pattern='<[^>]*>', repl='', string=str(title))
    content = re.sub(pattern='<[^>]*>', repl='', string=''.join(map(str, content)))
    content = content.replace("""[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}""", '')
    return title, content

# 뉴스 내용 추출
for url in tqdm(final_urls):
    title, content = extract_news_content(url)
    news_titles.append(title)
    news_contents.append(content)

# 결과 출력
def words_tokonizer(text):
    from konlpy.tag import Kkma # 한국어nlp
    kkma = Kkma()
    
    words = []
    
    
    s_list = text_preprocessor(text)
    
    # 태깅
    for s in s_list:
        words_ = kkma.pos(s)   
        
        # 인덱싱
        for word in words_:
            if word[1] == 'NNG':
                words.append(word[0])
            
    return words

# 결과 나옴
print("\n검색된 기사 수: ", len(news_titles))

for n, (title, content) in enumerate(zip(news_titles, news_contents), start=1):
    if title == None:
        continue
    print(f"\n{n}번 뉴스 제목: {words_tokonizer(title)}")
    print(f"뉴스 내용:\n{words_tokonizer(content)}\n")

생성된 URLs:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=축구&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=축구&start=11']


100%|██████████| 20/20 [00:07<00:00,  2.79it/s]



검색된 기사 수:  20

1번 뉴스 제목: ['축구', '배드민턴', '안', '양궁', '김우진', '항저우', '주목', '스타', '선정']
뉴스 내용:
['파리', '제', '르', '강인', '보루', '시', '챔피언', '스리', '경기', '출전', '모습', '로이터', '연합', '뉴스', '항저우', '아시안', '게임', '출격', '축구', '대표팀', '강인', '배드민턴', '안', '양궁', '김우진', '아시안', '게임', '조직', '위원회', '선정', '주목', '선수', '선정', '항저우', '아시안', '게임', '조직위', '공식', '개막', '사흘', '이번', '대회', '주목', '선수', '소개', '한국', '선수', '안', '김우진', '명단', '조직위', '강인', '한국', '월드컵', '결승', '행', '리', '메시', '포', '바', '골든', '볼', '소개', '올해', '세인', '스타', '군단', '이자', '프랑스', '리그', '챔피언', '파리', '생', '합류', '강인', '항저우', '입국', '남자', '축구', '조', '리그', '대표팀', '합류', '예정', '한국', '조', '리그', '차전', '파', '중국', '오픈', '우승', '인천', '공항', '귀국', '안', '뉴스', '안', '선', '세', '여자', '단식', '세계', '랭킹', '위', '한국', '여자', '선수', '최초', '세계', '선수권', '금메달', '소개', '조직위', '안', '올해', '차례', '투어', '대회', '우승', '세계', '챔피언', '일본', '야마', '구치', '올림픽', '금메달', '리스트', '중국', '상대', '승리', '야마', '구치', '안', '배드민턴', '여자', '단식', '천왕', '선수', '로', '지난해', '안', '선수', '약', '모습', '올해', '우위', '지난달', '서울', '목동', '